In [ ]:
# Libraries 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout
import tensorflow.keras as keras
import matplotlib.image as mpimg
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from scipy.signal import convolve2d
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Načtení csv informací o datech

In [ ]:
#importing training dataset
train=pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')
train_x=train['Path']
train_y=train.ClassId
class_num = len(train_y.drop_duplicates())

In [ ]:
#importing test dataset
test=pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')
test_x=test['Path']
test_y=test.ClassId
test_x

In [ ]:
data_dir = "../input/gtsrb-german-traffic-sign"
train_imgpath= list((data_dir + '/' + str(train.Path[i])) for i in range(len(train.Path)))
test_imgpath= list((data_dir + '/' + str(test.Path[i])) for i in range(len(test.Path)))

In [ ]:
Načtení a zobrazení testovacích a trenovacích dat

In [ ]:
plt.subplot(2,2,1)
im = Image.open(train_imgpath[0]) 
plt.imshow(im)
plt.subplot(2,2,2)
im = Image.open(train_imgpath[len(train_imgpath)-1]) 
plt.imshow(im)
plt.subplot(2,2,3)
im = Image.open(test_imgpath[0]) 
plt.imshow(im)
plt.subplot(2,2,4)
im = Image.open(test_imgpath[len(test_imgpath)-1]) 
plt.imshow(im)

In [ ]:
for i in range(0,18):
    plt.subplot(3,6,1+i)
    seed=np.random.randint(0,len(train_imgpath)-1)
    im = Image.open(train_imgpath[seed])  
    plt.imshow(im,aspect='auto')
plt.show()

In [ ]:
for i in range(0,18):
    plt.subplot(3,6,1+i)
    seed=np.random.randint(0,len(test_imgpath)-1)
    im = Image.open(test_imgpath[seed])  
    plt.imshow(im,aspect='auto')
plt.show()

změna velikosti obrazků na 32,32,3

In [ ]:
train_data=[]
train_labels=[]


path = "../input/gtsrb-german-traffic-sign/"
for i in range(len(train_imgpath)):
    image=cv2.imread(train_imgpath[i])
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((32,32))
    train_data.append(np.array(size_image))
    train_labels.append(train.ClassId[i])


Rozdělení dat na validační a trenovací

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split( np.array(train_data), np.array(train_labels), test_size=0.20, random_state=5151) 

In [ ]:
x_train = x_train.astype('float32')/255 
x_val =x_val.astype('float32')/255

#Using one hote encoding for the train and validation labels
from keras.utils import to_categorical
#y_train = to_categorical(y_train, class_num)
#y_val = to_categorical(y_val, class_num)

In [ ]:
test_data=[]
test_labels=[]


path = "../input/gtsrb-german-traffic-sign/"
for i in range(len(test.Path)):
    image=cv2.imread(test_imgpath[i])
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((32,32))
    test_data.append(np.array(size_image))
    test_labels.append(test.ClassId[i])


x_test=np.array(test_data)
y_test=np.array(test_labels)

x_test = x_test.astype('float32')/255 

In [ ]:
print('Train data shape: ', x_train.shape, y_train.shape)
print('Validation data shape: ', x_val.shape, y_val.shape)
print('Test data shape:  ', x_test.shape, y_test.shape)

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

In [ ]:
class Mish(keras.layers.Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

keras.utils.get_custom_objects().update({'mish': Mish(mish)})

Vytvoření modelu

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]),
    keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.15),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.15),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.15),
    keras.layers.Dense(class_num)
])

model.summary()

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, restore_best_weights=True)

history = model.fit(x_train, y_train, batch_size=32, validation_data=(x_val, y_val),callbacks=[es], epochs=50)
show_history(history)

test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy: ', test_acc)

In [ ]:
pred = model.predict_classes(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)